# Demand side flexibility


In [178]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("JuMP")
Pkg.add("Gurobi")
#Pkg.add("NLopt")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [179]:
using CSV, DataFrames, JuMP, Gurobi

# Read the CSV data

In [180]:
fixed_cost = Matrix(CSV.read("fixed_cost.csv",DataFrame))
variable_cost = Matrix(CSV.read("variable_cost.csv",DataFrame))
demand = Matrix(CSV.read("demand.csv", DataFrame))
Availability_matrix = Matrix(CSV.read("Availability.csv", DataFrame));

In [181]:
C_FC = fixed_cost[:,1]
C_FOM = fixed_cost[:,2]
C_VOM = variable_cost[:,1]
E_CO2 = variable_cost[:,2]
X_D = demand[1:10]
Availability = Availability_matrix[1:10,1:2]

10×2 Matrix{Float64}:
 0.00393802  0.53339
 0.0         0.513005
 0.0         0.518819
 0.0         0.481426
 0.0         0.475112
 0.0         0.469125
 0.0         0.468391
 0.0         0.453982
 0.0         0.411274
 0.0         0.376903

# Constant

In [182]:
eta = 0.8# Efficiency of the battery

 # price of the CO2 per ton

0.8

# Model Functions

In [184]:
function cost_mix(P, X)
    CO2_price = 1000000
    # The generation for the fossil fuel is for the index 3 and 4 of X
    print(sum((C_FC[i] + C_FOM[i])*P[i] for i in 1:size(P,1)))
    cost = sum((C_FC[i] + C_FOM[i])*P[i] for i in 1:size(P,1)) 
                sum(sum((C_VOM[j] + CO2_price * E_CO2[j])* X[t,j+2] for j in 1:2) for t in 1:size(X,1)) 
    return cost
end

cost_mix (generic function with 1 method)

# Creating the model

List of variable 

In [185]:
Power_generation_ls = 1:5 # Solar, Wind, CCGT, Coal, Battery

fossil_fuel_ls = 1:2 # CCGT, Coal

nb_hours_ls = 1:size(X_D,1)

H = 2

2

In [186]:
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "NonConvex", 2)
#model = Model(NLopt.Optimizer)
#set_optimizer_attribute(model, "algorithm", :LD_MMA)



#The installed power of the mix
@variable(model, P[Power_generation_ls] >= 0)

@constraint(model, Battery_power, P[5] <= P[1]+ P[2])
#@constraint(model, Battery_power_2, P[5] <= P[2])

#@constraint(model, Solar_power, P[1] <= 100000)
#@constraint(model, Wind_power, P[2] <= 100000)
#Generation of solar, wind, coal, CCGT
@variable(model, X_gen[nb_hours_ls, Power_generation_ls ] >= 0)

#State of charge of the battery
@variable(model, X_soc[nb_hours_ls] >=0)

# Discharge of the battery
@variable(model, X_dch[nb_hours_ls] >=0)

# Charge of the battery
@variable(model, X_ch[nb_hours_ls] >=0)

# The cost of the mix assuming a certain carbon cost
@objective(model, Min, cost_mix(P, X_gen)) # to be changed


# Cannot produce more than the installed power
@constraint(model, Solar_generation[t in nb_hours_ls], X_gen[t,1] <= P[1])
@constraint(model, Wind_generation[t in nb_hours_ls], X_gen[t,2] <= P[2])
@constraint(model, CCGT[t in nb_hours_ls], X_gen[t,3] <= P[3])
@constraint(model, Coal[t in nb_hours_ls], X_gen[t,4] <= P[4])
@constraint(model, Battery[t in nb_hours_ls], X_gen[t,5] == X_dch[t])

# Cannot produce more than availability for solar and wind 
@constraint(model, Solar_available[t in nb_hours_ls], X_gen[t,1] == Availability[t,1]*P[1])
@constraint(model, Wind_available[t in nb_hours_ls], X_gen[t,2] ==  Availability[t,2]*P[2])

# Supply meets demand at each hours:
@constraint(model, Supply_demand[t in nb_hours_ls], sum(X_gen[t,i] for i in Power_generation_ls) >= X_D[t]);


Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-07
Set parameter NonConvex to value 2
1.64493949138e6 P[1] + 695600.72477 P[2] + 1148000 P[3] + 3518500 P[4] + 434537.234 P[5]

![Alt text](image.png)

Expression for wind and solar

In [187]:
# Constrainhe state of charge
M = 100000

#Binary variable for the solar energy
@variable(model, s1[nb_hours_ls], Bin)

# z_1 equal 1 if eta*(x_g_t -x_d) is is greatter than eta*E_max*delta t
println("s_1 expression")
#@constraint(model, s_1_1[t in nb_hours_ls], Availability[t,1] - P[1]   <=  M*s1[t])
#@constraint(model, s_1_2[t in nb_hours_ls],  -Availability[t,1] + P[1] <= M*(1-s1[t]))

#@constraint(model, Solar_prod_1[t in nb_hours_ls], X_gen[t,1] >= P[1] -M*(1-s1[t]))
#@constraint(model, Solar_prod_2[t in nb_hours_ls], X_gen[t,1] >= Availability[t,1] -M*s1[t])


#Binary variable for the wind energy
@variable(model, s2[nb_hours_ls], Bin)

# z_1 equal 1 if eta*(x_g_t -x_d) is is greatter than eta*E_max*delta t
println("s_1 expression")
#@constraint(model, s_2_1[t in nb_hours_ls], Availability[t,2]- P[2]   <=  M*s2[t])
#@constraint(model, s_2_2[t in nb_hours_ls],  -Availability[t,2] + P[2] <= M*(1-s2[t]))

#@constraint(model, Wind_prod_1[t in nb_hours_ls], X_gen[t,2] >= P[2] -M*(1-s2[t]))
#@constraint(model, Wind_prod_2[t in nb_hours_ls], X_gen[t,2] >= Availability[t,2] -M*s2[t]);

s_1 expression
s_1 expression


Create the variable X_min = min(X_1, X_2, X_3)

In [188]:


# X_1, X_2, X_3 = [eta*(sum(X_gen[t,i] for i in Power_generation_ls)- X_D[t]) , eta*P[5], (1- X_soc[t])*P[5]* H]
@variable(model, z1[nb_hours_ls], Bin)
@variable(model, z2[nb_hours_ls], Bin)

# z_1 equal 1 if eta*(x_g_t -x_d) is is greatter than eta*E_max*delta t
println("Z_1 expression")
@constraint(model, z_1_1[t in nb_hours_ls], eta*P[5] - eta*(sum(X_gen[t,i] for i in 1:2)- X_D[t])   <=  M*z1[t])
@constraint(model, z_1_2[t in nb_hours_ls], -eta*P[5] + eta*(sum(X_gen[t,i] for i in 1:2)- X_D[t]) <= M*(1-z1[t]))


#Minimum between X_1 and X_2
@variable(model, min_X_1_X_2[nb_hours_ls])

println("Min X_1, X_2 expression")
@constraint(model, X_1_X_2_lower_1[t in nb_hours_ls], min_X_1_X_2[t] <= eta*(sum(X_gen[t,i] for i in 1:2)- X_D[t]))
@constraint(model, X_1_X_2_lower_2[t in nb_hours_ls], min_X_1_X_2[t] <= eta*P[5])
@constraint(model, X_1_X_2_higher_1[t in nb_hours_ls], min_X_1_X_2[t] >= eta*(sum(X_gen[t,i] for i in 1:2)- X_D[t]) - M*(1-z1[t]))
@constraint(model, X_1_X_2_higher_2[t in nb_hours_ls], min_X_1_X_2[t] >= eta*P[5] - M*z1[t])


# z_2 equal 1 if min(X_1, X_2) is is greater than (1- x_soc[t])*h*E_max
println(" Take the min of min(X_1, X_2) and X_3")
@constraint(model, z_2_1[t in nb_hours_ls], (1- X_soc[t])* P[5] * H - min_X_1_X_2[t] <=  M*z2[t])
@constraint(model, z_2_2[t in nb_hours_ls], - (1- X_soc[t]) * P[5] * H + min_X_1_X_2[t] <= M*(1-z2[t]))


# Constraint for the variable X_ch
println(" Take the overall min")
@variable(model, min_X_1_X_2_X_3[nb_hours_ls])

@constraint(model, X_1_X_2_X_3_lower_1[t in nb_hours_ls], min_X_1_X_2_X_3[t] <= min_X_1_X_2[t])
@constraint(model, X_1_X_2_X_3_lower_2[t in nb_hours_ls], min_X_1_X_2_X_3[t] <= (1- X_soc[t])*P[5]* H)
@constraint(model, X_1_X_2_X_3_higher_1[t in nb_hours_ls], min_X_1_X_2_X_3[t] >= min_X_1_X_2[t] - M*(1-z2[t]))
@constraint(model, X_1_X_2_X_3_higher_2[t in nb_hours_ls], min_X_1_X_2_X_3[t] >= (1- X_soc[t])*P[5]* H - M*z2[t]);



Z_1 expression
Min X_1, X_2 expression
 Take the min of min(X_1, X_2) and X_3
 Take the overall min


Expression of X_ch

In [189]:
@variable(model, Z_X_gen_greater_X_D[nb_hours_ls], Bin)

@constraint(model, Z_X_gen_greater_X_D_1[t in nb_hours_ls], sum(X_gen[t,i] for i in 1:2) - X_D[t] <= M*Z_X_gen_greater_X_D[t])
@constraint(model, Z_X_gen_greater_X_D_2[t in nb_hours_ls], -sum(X_gen[t,i] for i in 1:2) + X_D[t] <= M*(1-Z_X_gen_greater_X_D[t]))

#Expression of X_ch
@constraint(model, X_ch_expression[t in nb_hours_ls], X_ch[t]  == Z_X_gen_greater_X_D[t]*min_X_1_X_2_X_3[t]);

Expression of min of [X_gen[t]- X_D[t],  P[5], X_soc[t]*P[5]* H]

In [190]:
# X_1, X_2, X_3 = [X_gen[t]- X_D[t]) , P[5], X_soc[t])*P[5]* H]
@variable(model, y1[nb_hours_ls], Bin)
@variable(model, y2[nb_hours_ls], Bin)

# z_1 equal 1 if eta*(x_g_t -x_d) is is greatter than eta*E_max*delta t
println("Y_1 expression")
@constraint(model, y_1_1[t in nb_hours_ls], P[5] - (sum(X_gen[t,i] for i in 1:2)- X_D[t])   <=  M*y1[t])
@constraint(model, y_1_2[t in nb_hours_ls], -P[5] + (sum(X_gen[t,i] for i in 1:2)- X_D[t]) <= M*(1-y1[t]))


#Minimum between X_1 and X_2
@variable(model, min_extra_prod_P_5[nb_hours_ls])

println("Min X_1, X_2 expression")
@constraint(model, min_extra_prod_P_5_lower_1[t in nb_hours_ls], min_extra_prod_P_5[t] <= sum(X_gen[t,i] for i in 1:2)- X_D[t])
@constraint(model, min_extra_prod_P_5_lower_2[t in nb_hours_ls], min_extra_prod_P_5[t] <= P[5])
@constraint(model, min_extra_prod_P_5_higher_1[t in nb_hours_ls], min_extra_prod_P_5[t] >= (sum(X_gen[t,i] for i in 1:2)- X_D[t]) - M*(1-y1[t]))
@constraint(model, min_extra_prod_P_5_higher_2[t in nb_hours_ls], min_extra_prod_P_5[t] >= P[5] - M*y1[t])


# z_2 equal 1 if min(X_1, X_2) is is greater than (1- x_soc[t])*h*E_max
println(" Take the min of min(X_1, X_2) and X_3")
@constraint(model, y_2_1[t in nb_hours_ls], X_soc[t]* P[5] * H - min_extra_prod_P_5[t] <=  M*y2[t])
@constraint(model, y_2_2[t in nb_hours_ls], - X_soc[t] * P[5] * H + min_extra_prod_P_5[t] <= M*(1-y2[t]))


# Constraint for the variable X_ch
println(" Take the overall min")
@variable(model, min_extra_prod_P5_soc[nb_hours_ls])

@constraint(model, min_extra_prod_P5_soc_lower_1[t in nb_hours_ls], min_extra_prod_P5_soc[t] <= min_extra_prod_P_5[t])
@constraint(model, min_extra_prod_P5_soc_lower_2[t in nb_hours_ls], min_extra_prod_P5_soc[t] <= X_soc[t]*P[5]* H)
@constraint(model, min_extra_prod_P5_soc_higher_1[t in nb_hours_ls], min_extra_prod_P5_soc[t] >= min_extra_prod_P_5[t] - M*(1-y2[t]))
@constraint(model, min_extra_prod_P5_soc_higher_2[t in nb_hours_ls], min_extra_prod_P5_soc[t] >= X_soc[t]*P[5]* H - M*y2[t]);



Y_1 expression
Min X_1, X_2 expression
 Take the min of min(X_1, X_2) and X_3
 Take the overall min


Expression of X_dch

In [191]:
@variable(model, Z_X_gen_lower_X_D[nb_hours_ls], Bin)

@constraint(model, Z_X_gen_lower_X_D_1[t in nb_hours_ls], X_D[t] - sum(X_gen[t,i] for i in 1:4)  <= M*Z_X_gen_lower_X_D[t])
@constraint(model, Z_X_gen_lower_X_D_2[t in nb_hours_ls], -X_D[t] + sum(X_gen[t,i] for i in 1:4) <= M*(1-Z_X_gen_lower_X_D[t]))

#Expression of X_ch
@constraint(model, X_dch_expression[t in nb_hours_ls], X_dch[t]  == -Z_X_gen_lower_X_D[t]*min_extra_prod_P5_soc[t]);

Expression of state of charge

In [192]:
@constraint(model, SOC_t_1, X_soc[1] == 1)

@constraint(model, SOC_t[t in nb_hours_ls[2:end]], (X_soc[t]-X_soc[t-1])*H*P[5] == (X_ch[t-1]- X_dch[t-1]));

In [193]:
optimize!(model);

Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 282 rows, 205 columns and 845 nonzeros
Model fingerprint: 0x7d76d0ee
Model has 109 quadratic constraints
Variable types: 125 continuous, 80 integer (80 binary)
Coefficient statistics:
  Matrix range     [4e-03, 1e+05]
  QMatrix range    [1e+00, 2e+00]
  QLMatrix range   [1e+00, 1e+05]
  Objective range  [4e+05, 4e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 6e+05]
  QRHS range       [1e+05, 1e+05]
Presolve removed 169 rows and 131 columns
Presolve time: 0.00s
Presolved: 205 rows, 87 columns, 506 nonzeros
Presolved model has 25 bilinear constraint(s)
Variable types: 68 continuous, 19 integer (19 binary)

Root relaxation: objective 7.147005e+11, 63 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |

In [194]:
objective_value(model)

7.628401327226421e11

In [195]:
cost_mix(value.(P), value.(X_gen))

7.628401327226421e11

7.628401327226421e11

In [196]:
println(value.(P))
println(value.(s1[:]))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:5
And data, a 5-element Vector{Float64}:
      0.0
 998491.4207202638
  59484.997206445725
      0.0
      0.0
1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0


In [197]:
println(value.(X_gen))

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:10
    Dimension 2, 1:5
And data, a 10×5 Matrix{Float64}:
 0.0  532585.779232698        0.0             0.0      0.0
 0.0  512231.342906437        0.0             0.0      0.0
 0.0  518036.36933274614      0.0             0.0      0.0
 0.0  480700.1281112592       0.0             0.0      0.0
 0.0  474395.5843849234       0.0             0.0      0.0
 0.0  468417.15394754335      0.0             0.0      0.0
 0.0  467684.6207016462       0.0             0.0      0.0
 0.0  453297.28992307134  16411.853776928692  0.0      0.0
 0.0  410653.9389935543   59484.997206445725  0.0      0.0
 0.0  376334.9042             0.0             0.0  99999.99999999999


In [198]:
println(value.(Z_X_gen_greater_X_D))
println(value.(X_soc))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{Float64}:
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
 -8.866710260862502e-16
  0.0
  0.0
1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{Float64}:
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0


In [201]:
println(value.(X_ch))
println(value.(X_dch))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{Float64}:
     0.0
     0.0
     0.0
     0.0
     0.0
     0.0
     0.0
     0.0
     0.0
 99999.99999999999


In [203]:
println(value.((sum(X_gen[:,i] for i in 1:2)- X_D[:]) ))
println(value.(P[5]))
println(value.(min_extra_prod_P_5[:]))
println(value.(X_soc[:]* P[5] * H))
println(value.(min_extra_prod_P5_soc))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{Float64}:
   74856.86133269803
   63282.775406436995
   73820.36893274612
   40321.81981125922
   32038.084784923412
   17911.032047543325
    1418.786301646207
  -16411.853776928678
  -59484.997206445725
 -100000.0
0.0
1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{Float64}:
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
 -16411.853776928692
 -59484.997206445725
 -99999.99999999999
1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{Float64}:
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
 -16411.853776928692
 -59484.997206445725
 -99

In [205]:
println(value.(Z_X_gen_lower_X_D))
println(value.(-X_D[:] + sum(X_gen[:,i] for i in 1:4)))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{Float64}:
   74856.86133269803
   63282.775406436995
   73820.36893274612
   40321.81981125922
   32038.084784923412
   17911.032047543325
    1418.786301646207
       1.4551915228366852e-11
       0.0
 -100000.0
